Project aims to solve MLC scheduling problem:
- k number of instructors with workload wk (number of hours to be scheduled at the MLC per week)
- Only one instructor can be scheduled at a time
- Shifts are half-hour segments; instructors work several segments consecutively if they are available

first, we'll install and import the PuLP open source library, and instantiate the model:

In [ ]:
!pip install pulp
import pulp

mlc_scheduling_model = pulp.LpProblem("mlc_schedule", pulp.LpMaximize)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 32.2 MB/s eta 0:00:00


We now define our variables, starting with instructors. Each instructor has an assigned weekly workload of hours in the MLC.

In [ ]:
y = ['AA', 'MA', 'MA2', 'DB', 'LL', 'PO', 'DW', 'LY', 'Dummy']
workload = [0.5, 6, 4.5, 2, 6, 3, 5, 6, 4.5]

Next, shifts. Here, i represents days of the week, and j represents the index of a certain half-hour shift segment, starting with 9am = 0, 9:30 = 1, etc.

In [ ]:
i = ['M', 'T', 'W', 'R', 'F']
j = ['9:00', '9:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '1:00', '1:30',
     '2:00', '2:30', '3:00', '3:30', '4:00']

x = []
for index_1 in range(5):
  nested_list = []
  for index_2 in range(len(j)):
    nested_list.append(i[index_1] + j[index_2])
  x.append(nested_list)

print(x)
print(x[2][0])

[['M9:00', 'M9:30', 'M10:00', 'M10:30', 'M11:00', 'M11:30', 'M12:00', 'M12:30', 'M1:00', 'M1:30', 'M2:00', 'M2:30', 'M3:00', 'M3:30', 'M4:00'], ['T9:00', 'T9:30', 'T10:00', 'T10:30', 'T11:00', 'T11:30', 'T12:00', 'T12:30', 'T1:00', 'T1:30', 'T2:00', 'T2:30', 'T3:00', 'T3:30', 'T4:00'], ['W9:00', 'W9:30', 'W10:00', 'W10:30', 'W11:00', 'W11:30', 'W12:00', 'W12:30', 'W1:00', 'W1:30', 'W2:00', 'W2:30', 'W3:00', 'W3:30', 'W4:00'], ['R9:00', 'R9:30', 'R10:00', 'R10:30', 'R11:00', 'R11:30', 'R12:00', 'R12:30', 'R1:00', 'R1:30', 'R2:00', 'R2:30', 'R3:00', 'R3:30', 'R4:00'], ['F9:00', 'F9:30', 'F10:00', 'F10:30', 'F11:00', 'F11:30', 'F12:00', 'F12:30', 'F1:00', 'F1:30', 'F2:00', 'F2:30', 'F3:00', 'F3:30', 'F4:00']]
W9:00


The model gives the user the opportunity to hardcode instructors' preferences into the model, or enter the data manually. Here is the hardcoded data from Fall 2023, based off the schedule shown below (I took some liberties in assigning instructors' preferred times, mostly just assuming no one would want to be on campus too late in day or on their days off.) put schedule here

In [ ]:
#y = ['AA', 'MA', 'MA2', 'DB', 'LL', 'PO', 'DW', 'LY', 'Dummy']
# i = ['M', 'T', 'W', 'R', 'F']
# j = ['9:00', '9:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '1:00', '1:30',
#      '2:00', '2:30', '3:00', '3:30', '4:00']
# numeric scale:
# 0 - not available
# 1 - strongly preferred not - days off, coming in early/staying late etc
# 2 - can work if no one else available
# 3 - ok with me
# 4 - preferred
# 5 - prioritize me over other instructors


hardcoded_preferences = {(x[0][0], y[0]): 1, (x[0][1], y[0]): 1, (x[0][2], y[0]): 3, (x[0][3], y[0]): 3, (x[0][4], y[0]): 3,
                         (x[0][5], y[0]): 3, (x[0][6], y[0]): 3, (x[0][7], y[0]): 3, (x[0][8], y[0]): 4, (x[0][9], y[0]): 4,
                         (x[0][10], y[0]): 4, (x[0][11], y[0]): 0, (x[0][12], y[0]): 0,(x[0][13], y[0]): 0, (x[0][14], y[0]): 1,

                          (x[1][0], y[0]): 1, (x[1][1], y[0]): 1, (x[1][2], y[0]): 1, (x[1][3], y[0]): 1, (x[1][4], y[0]): 1,
                         (x[1][5], y[0]): 0, (x[1][6], y[0]): 0, (x[1][7], y[0]): 0, (x[1][8], y[0]): 1, (x[1][9], y[0]): 1,
                         (x[1][10], y[0]): 1, (x[1][11], y[0]): 1, (x[1][12], y[0]): 1,(x[1][13], y[0]): 1, (x[1][14], y[0]): 1,

                         (x[2][0], y[0]): 1, (x[2][1], y[0]): 1, (x[2][2], y[0]): 3, (x[2][3], y[0]): 3, (x[2][4], y[0]): 3,
                         (x[2][5], y[0]): 3, (x[2][6], y[0]): 3, (x[2][7], y[0]): 3, (x[2][8], y[0]): 4, (x[2][9], y[0]): 4,
                         (x[2][10], y[0]): 4, (x[2][11], y[0]): 0, (x[2][12], y[0]): 0,(x[2][13], y[0]): 0, (x[2][14], y[0]): 1,

                         (x[3][0], y[0]): 1, (x[3][1], y[0]): 1, (x[3][2], y[0]): 1, (x[3][3], y[0]): 1, (x[3][4], y[0]): 1,
                         (x[3][5], y[0]): 0, (x[3][6], y[0]): 0, (x[3][7], y[0]): 0, (x[3][8], y[0]): 1, (x[3][9], y[0]): 1,
                         (x[3][10], y[0]): 1, (x[3][11], y[0]): 1, (x[3][12], y[0]): 1,(x[3][13], y[0]): 1, (x[3][14], y[0]): 1,

                         (x[4][0], y[0]): 1, (x[4][1], y[0]): 1, (x[4][2], y[0]): 1, (x[4][3], y[0]): 1, (x[4][4], y[0]): 1,
                         (x[4][5], y[0]): 1, (x[4][6], y[0]): 1, (x[4][7], y[0]): 1, (x[4][8], y[0]): 1, (x[4][9], y[0]): 1,
                         (x[4][10], y[0]): 1, (x[4][11], y[0]): 1, (x[4][12], y[0]): 0,(x[4][13], y[0]): 0, (x[4][14], y[0]): 0,



                          (x[0][0], y[1]): 3, (x[0][1], y[1]): 3, (x[0][2], y[1]): 3, (x[0][3], y[1]): 2, (x[0][4], y[1]): 2,
                         (x[0][5], y[1]): 0, (x[0][6], y[1]): 0, (x[0][7], y[1]): 0, (x[0][8], y[1]): 0, (x[0][9], y[1]): 0,
                         (x[0][10], y[1]): 0, (x[0][11], y[1]): 2, (x[0][12], y[1]): 2,(x[0][13], y[1]): 2, (x[0][14], y[1]): 2,

                          (x[1][0], y[1]): 0, (x[1][1], y[1]): 0, (x[1][2], y[1]): 3, (x[1][3], y[1]): 3, (x[1][4], y[1]): 3,
                         (x[1][5], y[1]): 0, (x[1][6], y[1]): 0, (x[1][7], y[1]): 0, (x[1][8], y[1]): 1, (x[1][9], y[1]): 1,
                         (x[1][10], y[1]): 3, (x[1][11], y[1]): 3, (x[1][12], y[1]): 3,(x[1][13], y[1]): 1, (x[1][14], y[1]): 1,

                         (x[2][0], y[1]): 4, (x[2][1], y[1]): 4, (x[2][2], y[1]): 4, (x[2][3], y[1]): 4, (x[2][4], y[1]): 4,
                         (x[2][5], y[1]): 0, (x[2][6], y[1]): 0, (x[2][7], y[1]): 0, (x[2][8], y[1]): 2, (x[2][9], y[1]): 2,
                         (x[2][10], y[1]): 4, (x[2][11], y[1]): 4, (x[2][12], y[1]): 4,(x[2][13], y[1]): 2, (x[2][14], y[1]): 2,

                         (x[3][0], y[1]): 0, (x[3][1], y[1]): 0, (x[3][2], y[1]): 3, (x[3][3], y[1]): 3, (x[3][4], y[1]): 3,
                         (x[3][5], y[1]): 0, (x[3][6], y[1]): 0, (x[3][7], y[1]): 0, (x[3][8], y[1]): 1, (x[3][9], y[1]): 1,
                         (x[3][10], y[1]): 3, (x[3][11], y[1]): 3, (x[3][12], y[1]): 3,(x[3][13], y[1]): 1, (x[3][14], y[1]): 1,

                         (x[4][0], y[1]): 0, (x[4][1], y[1]): 0, (x[4][2], y[1]): 4, (x[4][3], y[1]): 4, (x[4][4], y[1]): 4,
                         (x[4][5], y[1]): 4, (x[4][6], y[1]): 4, (x[4][7], y[1]): 4, (x[4][8], y[1]): 4, (x[4][9], y[1]): 4,
                         (x[4][10], y[1]): 4, (x[4][11], y[1]): 3, (x[4][12], y[1]): 0,(x[4][13], y[1]): 0, (x[4][14], y[1]): 0,



                          (x[0][0], y[2]): 2, (x[0][1], y[2]): 2, (x[0][2], y[2]): 4, (x[0][3], y[2]): 4, (x[0][4], y[2]): 4,
                         (x[0][5], y[2]): 0, (x[0][6], y[2]): 0, (x[0][7], y[2]): 0, (x[0][8], y[2]): 1, (x[0][9], y[2]): 1,
                         (x[0][10], y[2]): 3, (x[0][11], y[2]): 3, (x[0][12], y[2]): 3,(x[0][13], y[2]): 1, (x[0][14], y[2]): 1,

                          (x[1][0], y[2]): 0, (x[1][1], y[2]): 0, (x[1][2], y[2]): 0, (x[1][3], y[2]): 0, (x[1][4], y[2]): 0,
                         (x[1][5], y[2]): 0, (x[1][6], y[2]): 0, (x[1][7], y[2]): 0, (x[1][8], y[2]): 2, (x[1][9], y[2]): 2,
                         (x[1][10], y[2]): 3, (x[1][11], y[2]): 3, (x[1][12], y[2]): 1,(x[1][13], y[2]): 1, (x[1][14], y[2]): 1,

                         (x[2][0], y[2]): 1, (x[2][1], y[2]): 1, (x[2][2], y[2]): 1, (x[2][3], y[2]): 1, (x[2][4], y[2]): 1,
                         (x[2][5], y[2]): 1, (x[2][6], y[2]): 1, (x[2][7], y[2]): 1, (x[2][8], y[2]): 1, (x[2][9], y[2]): 1,
                         (x[2][10], y[2]): 1, (x[2][11], y[2]): 1, (x[2][12], y[2]): 1,(x[2][13], y[2]): 1, (x[2][14], y[2]): 1,

                         (x[3][0], y[2]): 0, (x[3][1], y[2]): 0, (x[3][2], y[2]): 4, (x[3][3], y[2]): 4, (x[3][4], y[2]): 4,
                         (x[3][5], y[2]): 0, (x[3][6], y[2]): 0, (x[3][7], y[2]): 0, (x[3][8], y[2]): 2, (x[3][9], y[2]): 2,
                         (x[3][10], y[2]): 3, (x[3][11], y[2]): 3, (x[3][12], y[2]): 1,(x[3][13], y[2]): 1, (x[3][14], y[2]): 1,

                         (x[4][0], y[2]): 2, (x[4][1], y[2]): 2, (x[4][2], y[2]): 0, (x[4][3], y[2]): 0, (x[4][4], y[2]): 0,
                         (x[4][5], y[2]): 4, (x[4][6], y[2]): 4, (x[4][7], y[2]): 4, (x[4][8], y[2]): 4, (x[4][9], y[2]): 4,
                         (x[4][10], y[2]): 4, (x[4][11], y[2]): 3, (x[4][12], y[2]): 0,(x[4][13], y[2]): 0, (x[4][14], y[2]): 0,



                         (x[0][0], y[3]): 3, (x[0][1], y[3]): 3, (x[0][2], y[3]): 3, (x[0][3], y[3]): 3, (x[0][4], y[3]): 4,
                         (x[0][5], y[3]): 4, (x[0][6], y[3]): 4, (x[0][7], y[3]): 4, (x[0][8], y[3]): 0, (x[0][9], y[3]): 0,
                         (x[0][10], y[3]): 0, (x[0][11], y[3]): 4, (x[0][12], y[3]): 4,(x[0][13], y[3]): 4, (x[0][14], y[3]): 4,

                          (x[1][0], y[3]): 1, (x[1][1], y[3]): 1, (x[1][2], y[3]): 1, (x[1][3], y[3]): 1, (x[1][4], y[3]): 1,
                         (x[1][5], y[3]): 0, (x[1][6], y[3]): 0, (x[1][7], y[3]): 0, (x[1][8], y[3]): 3, (x[1][9], y[3]): 3,
                         (x[1][10], y[3]): 4, (x[1][11], y[3]): 4, (x[1][12], y[3]): 4,(x[1][13], y[3]): 4, (x[1][14], y[3]): 4,

                         (x[2][0], y[3]): 1, (x[2][1], y[3]): 1, (x[2][2], y[3]): 1, (x[2][3], y[3]): 1, (x[2][4], y[3]): 1,
                         (x[2][5], y[3]): 1, (x[2][6], y[3]): 1, (x[2][7], y[3]): 1, (x[2][8], y[3]): 1, (x[2][9], y[3]): 1,
                         (x[2][10], y[3]): 1, (x[2][11], y[3]): 1, (x[2][12], y[3]): 1,(x[2][13], y[3]): 1, (x[2][14], y[3]): 1,

                         (x[3][0], y[3]): 3, (x[3][1], y[3]): 3, (x[3][2], y[3]): 0, (x[3][3], y[3]): 0, (x[3][4], y[3]): 0,
                         (x[3][5], y[3]): 0, (x[3][6], y[3]): 0, (x[3][7], y[3]): 0, (x[3][8], y[3]): 4, (x[3][9], y[3]): 4,
                         (x[3][10], y[3]): 4, (x[3][11], y[3]): 4, (x[3][12], y[3]): 3,(x[3][13], y[3]): 3, (x[3][14], y[3]): 3,

                         (x[4][0], y[3]): 1, (x[4][1], y[3]): 1, (x[4][2], y[3]): 1, (x[4][3], y[3]): 1, (x[4][4], y[3]): 1,
                         (x[4][5], y[3]): 1, (x[4][6], y[3]): 1, (x[4][7], y[3]): 1, (x[4][8], y[3]): 1, (x[4][9], y[3]): 1,
                         (x[4][10], y[3]): 1, (x[4][11], y[3]): 1, (x[4][12], y[3]): 0,(x[4][13], y[3]): 0, (x[4][14], y[3]): 0,



                         (x[0][0], y[4]): 0, (x[0][1], y[4]): 0, (x[0][2], y[4]): 3, (x[0][3], y[4]): 3, (x[0][4], y[4]): 3,
                         (x[0][5], y[4]): 0, (x[0][6], y[4]): 0, (x[0][7], y[4]): 0, (x[0][8], y[4]): 4, (x[0][9], y[4]): 4,
                         (x[0][10], y[4]): 4, (x[0][11], y[4]): 4, (x[0][12], y[4]): 3,(x[0][13], y[4]): 3, (x[0][14], y[4]): 3,

                          (x[1][0], y[4]): 2, (x[1][1], y[4]): 2, (x[1][2], y[4]): 2, (x[1][3], y[4]): 2, (x[1][4], y[4]): 2,
                         (x[1][5], y[4]): 0, (x[1][6], y[4]): 0, (x[1][7], y[4]): 0, (x[1][8], y[4]): 2, (x[1][9], y[4]): 2,
                         (x[1][10], y[4]): 2, (x[1][11], y[4]): 2, (x[1][12], y[4]): 2,(x[1][13], y[4]): 2, (x[1][14], y[4]): 2,

                         (x[2][0], y[4]): 0, (x[2][1], y[4]): 0, (x[2][2], y[4]): 2, (x[2][3], y[4]): 3, (x[2][4], y[4]): 3,
                         (x[2][5], y[4]): 0, (x[2][6], y[4]): 0, (x[2][7], y[4]): 0, (x[2][8], y[4]): 4, (x[2][9], y[4]): 4,
                         (x[2][10], y[4]): 4, (x[2][11], y[4]): 4, (x[2][12], y[4]): 3,(x[2][13], y[4]): 3, (x[2][14], y[4]): 3,

                         (x[3][0], y[4]): 2, (x[3][1], y[4]): 2, (x[3][2], y[4]): 2, (x[3][3], y[4]): 2, (x[3][4], y[4]): 2,
                         (x[3][5], y[4]): 0, (x[3][6], y[4]): 0, (x[3][7], y[4]): 0, (x[3][8], y[4]): 2, (x[3][9], y[4]): 2,
                         (x[3][10], y[4]): 2, (x[3][11], y[4]): 2, (x[3][12], y[4]): 2,(x[3][13], y[4]): 2, (x[3][14], y[4]): 2,

                         (x[4][0], y[4]): 0, (x[4][1], y[4]): 0, (x[4][2], y[4]): 3, (x[4][3], y[4]): 3, (x[4][4], y[4]): 3,
                         (x[4][5], y[4]): 0, (x[4][6], y[4]): 0, (x[4][7], y[4]): 0, (x[4][8], y[4]): 4, (x[4][9], y[4]): 4,
                         (x[4][10], y[4]): 4, (x[4][11], y[4]): 4, (x[4][12], y[4]): 0,(x[4][13], y[4]): 0, (x[4][14], y[4]): 0,



                         (x[0][0], y[5]): 1, (x[0][1], y[5]): 1, (x[0][2], y[5]): 0, (x[0][3], y[5]): 0, (x[0][4], y[5]): 0,
                         (x[0][5], y[5]): 0, (x[0][6], y[5]): 0, (x[0][7], y[5]): 0, (x[0][8], y[5]): 3, (x[0][9], y[5]): 3,
                         (x[0][10], y[5]): 4, (x[0][11], y[5]): 4, (x[0][12], y[5]): 4,(x[0][13], y[5]): 3, (x[0][14], y[5]): 0,

                          (x[1][0], y[5]): 1, (x[1][1], y[5]): 1, (x[1][2], y[5]): 1, (x[1][3], y[5]): 1, (x[1][4], y[5]): 1,
                         (x[1][5], y[5]): 0, (x[1][6], y[5]): 0, (x[1][7], y[5]): 0, (x[1][8], y[5]): 1, (x[1][9], y[5]): 1,
                         (x[1][10], y[5]): 1, (x[1][11], y[5]): 1, (x[1][12], y[5]): 1,(x[1][13], y[5]): 1, (x[1][14], y[5]): 1,

                         (x[2][0], y[5]): 1, (x[2][1], y[5]): 1, (x[2][2], y[5]): 0, (x[2][3], y[5]): 0, (x[2][4], y[5]): 0,
                         (x[2][5], y[5]): 0, (x[2][6], y[5]): 0, (x[2][7], y[5]): 0, (x[2][8], y[5]): 2, (x[2][9], y[5]): 2,
                         (x[2][10], y[5]): 3, (x[2][11], y[5]): 3, (x[2][12], y[5]): 2,(x[2][13], y[5]): 2, (x[2][14], y[5]): 0,

                         (x[3][0], y[5]): 1, (x[3][1], y[5]): 1, (x[3][2], y[5]): 1, (x[3][3], y[5]): 1, (x[3][4], y[5]): 1,
                         (x[3][5], y[5]): 0, (x[3][6], y[5]): 0, (x[3][7], y[5]): 0, (x[3][8], y[5]): 1, (x[3][9], y[5]): 1,
                         (x[3][10], y[5]): 1, (x[3][11], y[5]): 1, (x[3][12], y[5]): 1,(x[3][13], y[5]): 1, (x[3][14], y[5]): 1,

                         (x[4][0], y[5]): 2, (x[4][1], y[5]): 2, (x[4][2], y[5]): 0, (x[4][3], y[5]): 0, (x[4][4], y[5]): 0,
                         (x[4][5], y[5]): 0, (x[4][6], y[5]): 0, (x[4][7], y[5]): 0, (x[4][8], y[5]): 3, (x[4][9], y[5]): 3,
                         (x[4][10], y[5]): 4, (x[4][11], y[5]): 4, (x[4][12], y[5]): 0,(x[4][13], y[5]): 0, (x[4][14], y[5]): 0,



                         (x[0][0], y[6]): 0, (x[0][1], y[6]): 0, (x[0][2], y[6]): 4, (x[0][3], y[6]): 4, (x[0][4], y[6]): 4,
                         (x[0][5], y[6]): 4, (x[0][6], y[6]): 3, (x[0][7], y[6]): 3, (x[0][8], y[6]): 3, (x[0][9], y[6]): 1,
                         (x[0][10], y[6]): 1, (x[0][11], y[6]): 1, (x[0][12], y[6]): 1,(x[0][13], y[6]): 1, (x[0][14], y[6]): 1,

                          (x[1][0], y[6]): 1, (x[1][1], y[6]): 1, (x[1][2], y[6]): 1, (x[1][3], y[6]): 1, (x[1][4], y[6]): 1,
                         (x[1][5], y[6]): 0, (x[1][6], y[6]): 0, (x[1][7], y[6]): 0, (x[1][8], y[6]): 2, (x[1][9], y[6]): 2,
                         (x[1][10], y[6]): 4, (x[1][11], y[6]): 4, (x[1][12], y[6]): 4,(x[1][13], y[6]): 4, (x[1][14], y[6]): 0,

                         (x[2][0], y[6]): 0, (x[2][1], y[6]): 0, (x[2][2], y[6]): 4, (x[2][3], y[6]): 4, (x[2][4], y[6]): 4,
                         (x[2][5], y[6]): 4, (x[2][6], y[6]): 3, (x[2][7], y[6]): 3, (x[2][8], y[6]): 3, (x[2][9], y[6]): 1,
                         (x[2][10], y[6]): 1, (x[2][11], y[6]): 1, (x[2][12], y[6]): 1,(x[2][13], y[6]): 1, (x[2][14], y[6]): 1,

                         (x[3][0], y[6]): 1, (x[3][1], y[6]): 1, (x[3][2], y[6]): 1, (x[3][3], y[6]): 1, (x[3][4], y[6]): 1,
                         (x[3][5], y[6]): 0, (x[3][6], y[6]): 0, (x[3][7], y[6]): 0, (x[3][8], y[6]): 2, (x[3][9], y[6]): 2,
                         (x[3][10], y[6]): 4, (x[3][11], y[6]): 4, (x[3][12], y[6]): 4,(x[3][13], y[6]): 4, (x[3][14], y[6]): 0,

                         (x[4][0], y[6]): 1, (x[4][1], y[6]): 1, (x[4][2], y[6]): 1, (x[4][3], y[6]): 1, (x[4][4], y[6]): 1,
                         (x[4][5], y[6]): 1, (x[4][6], y[6]): 1, (x[4][7], y[6]): 1, (x[4][8], y[6]): 1, (x[4][9], y[6]): 1,
                         (x[4][10], y[6]): 1, (x[4][11], y[6]): 1, (x[4][12], y[6]): 0,(x[4][13], y[6]): 0, (x[4][14], y[6]): 0,



                         (x[0][0], y[7]): 4, (x[0][1], y[7]): 4, (x[0][2], y[7]): 0, (x[0][3], y[7]): 0, (x[0][4], y[7]): 0,
                         (x[0][5], y[7]): 4, (x[0][6], y[7]): 4, (x[0][7], y[7]): 4, (x[0][8], y[7]): 4, (x[0][9], y[7]): 2,
                         (x[0][10], y[7]): 2, (x[0][11], y[7]): 1, (x[0][12], y[7]): 1,(x[0][13], y[7]): 1, (x[0][14], y[7]): 1,

                          (x[1][0], y[7]): 1, (x[1][1], y[7]): 1, (x[1][2], y[7]): 1, (x[1][3], y[7]): 1, (x[1][4], y[7]): 1,
                         (x[1][5], y[7]): 0, (x[1][6], y[7]): 0, (x[1][7], y[7]): 0, (x[1][8], y[7]): 1, (x[1][9], y[7]): 1,
                         (x[1][10], y[7]): 1, (x[1][11], y[7]): 1, (x[1][12], y[7]): 1,(x[1][13], y[7]): 1, (x[1][14], y[7]): 1,

                         (x[2][0], y[7]): 4, (x[2][1], y[7]): 4, (x[2][2], y[7]): 0, (x[2][3], y[7]): 0, (x[2][4], y[7]): 0,
                         (x[2][5], y[7]): 4, (x[2][6], y[7]): 4, (x[2][7], y[7]): 4, (x[2][8], y[7]): 4, (x[2][9], y[7]): 2,
                         (x[2][10], y[7]): 2, (x[2][11], y[7]): 1, (x[2][12], y[7]): 1,(x[2][13], y[7]): 1, (x[2][14], y[7]): 1,

                         (x[3][0], y[7]): 1, (x[3][1], y[7]): 1, (x[3][2], y[7]): 1, (x[3][3], y[7]): 1, (x[3][4], y[7]): 1,
                         (x[3][5], y[7]): 0, (x[3][6], y[7]): 0, (x[3][7], y[7]): 0, (x[3][8], y[7]): 1, (x[3][9], y[7]): 1,
                         (x[3][10], y[7]): 1, (x[3][11], y[7]): 1, (x[3][12], y[7]): 1,(x[3][13], y[7]): 1, (x[3][14], y[7]): 1,

                         (x[4][0], y[7]): 1, (x[4][1], y[7]): 1, (x[4][2], y[7]): 1, (x[4][3], y[7]): 1, (x[4][4], y[7]): 1,
                         (x[4][5], y[7]): 1, (x[4][6], y[7]): 1, (x[4][7], y[7]): 1, (x[4][8], y[7]): 1, (x[4][9], y[7]): 1,
                         (x[4][10], y[7]): 1, (x[4][11], y[7]): 1, (x[4][12], y[7]): 0,(x[4][13], y[7]): 0, (x[4][14], y[7]): 0,



                         (x[0][0], y[8]): 0, (x[0][1], y[8]): 0, (x[0][2], y[8]): 0, (x[0][3], y[8]): 0, (x[0][4], y[8]): 0,
                         (x[0][5], y[8]): 0, (x[0][6], y[8]): 0, (x[0][7], y[8]): 0, (x[0][8], y[8]): 0, (x[0][9], y[8]): 0,
                         (x[0][10], y[8]): 0, (x[0][11], y[8]): 0, (x[0][12], y[8]): 0,(x[0][13], y[8]): 0, (x[0][14], y[8]): 0,

                          (x[1][0], y[8]): 0, (x[1][1], y[8]): 0, (x[1][2], y[8]): 0, (x[1][3], y[8]): 0, (x[1][4], y[8]): 0,
                         (x[1][5], y[8]): 5, (x[1][6], y[8]): 5, (x[1][7], y[8]): 5, (x[1][8], y[8]): 0, (x[1][9], y[8]): 0,
                         (x[1][10], y[8]): 0, (x[1][11], y[8]): 0, (x[1][12], y[8]): 0,(x[1][13], y[8]): 0, (x[1][14], y[8]): 0,

                         (x[2][0], y[8]): 0, (x[2][1], y[8]): 0, (x[2][2], y[8]): 0, (x[2][3], y[8]): 0, (x[2][4], y[8]): 0,
                         (x[2][5], y[8]): 0, (x[2][6], y[8]): 0, (x[2][7], y[8]): 0, (x[2][8], y[8]): 0, (x[2][9], y[8]): 0,
                         (x[2][10], y[8]): 0, (x[2][11], y[8]): 0, (x[2][12], y[8]): 0,(x[2][13], y[8]): 0, (x[2][14], y[8]): 0,

                         (x[3][0], y[8]): 0, (x[3][1], y[8]): 0, (x[3][2], y[8]): 0, (x[3][3], y[8]): 0, (x[3][4], y[8]): 0,
                         (x[3][5], y[8]): 5, (x[3][6], y[8]): 5, (x[3][7], y[8]): 5, (x[3][8], y[8]): 0, (x[3][9], y[8]): 0,
                         (x[3][10], y[8]): 0, (x[3][11], y[8]): 0, (x[3][12], y[8]): 0,(x[3][13], y[8]): 0, (x[3][14], y[8]): 0,

                         (x[4][0], y[8]): 0, (x[4][1], y[8]): 0, (x[4][2], y[8]): 0, (x[4][3], y[8]): 0, (x[4][4], y[8]): 0,
                         (x[4][5], y[8]): 0, (x[4][6], y[8]): 0, (x[4][7], y[8]): 0, (x[4][8], y[8]): 0, (x[4][9], y[8]): 0,
                         (x[4][10], y[8]): 0, (x[4][11], y[8]): 0, (x[4][12], y[8]): 5,(x[4][13], y[8]): 5, (x[4][14], y[8]): 5}
print(hardcoded_preferences)

{('M9:00', 'AA'): 1, ('M9:30', 'AA'): 1, ('M10:00', 'AA'): 3, ('M10:30', 'AA'): 3, ('M11:00', 'AA'): 3, ('M11:30', 'AA'): 3, ('M12:00', 'AA'): 3, ('M12:30', 'AA'): 3, ('M1:00', 'AA'): 4, ('M1:30', 'AA'): 4, ('M2:00', 'AA'): 4, ('M2:30', 'AA'): 0, ('M3:00', 'AA'): 0, ('M3:30', 'AA'): 0, ('M4:00', 'AA'): 1, ('T9:00', 'AA'): 1, ('T9:30', 'AA'): 1, ('T10:00', 'AA'): 1, ('T10:30', 'AA'): 1, ('T11:00', 'AA'): 1, ('T11:30', 'AA'): 0, ('T12:00', 'AA'): 0, ('T12:30', 'AA'): 0, ('T1:00', 'AA'): 1, ('T1:30', 'AA'): 1, ('T2:00', 'AA'): 1, ('T2:30', 'AA'): 1, ('T3:00', 'AA'): 1, ('T3:30', 'AA'): 1, ('T4:00', 'AA'): 1, ('W9:00', 'AA'): 1, ('W9:30', 'AA'): 1, ('W10:00', 'AA'): 3, ('W10:30', 'AA'): 3, ('W11:00', 'AA'): 3, ('W11:30', 'AA'): 3, ('W12:00', 'AA'): 3, ('W12:30', 'AA'): 3, ('W1:00', 'AA'): 4, ('W1:30', 'AA'): 4, ('W2:00', 'AA'): 4, ('W2:30', 'AA'): 0, ('W3:00', 'AA'): 0, ('W3:30', 'AA'): 0, ('W4:00', 'AA'): 1, ('R9:00', 'AA'): 1, ('R9:30', 'AA'): 1, ('R10:00', 'AA'): 1, ('R10:30', 'AA'): 1,

And here is the part where the user can enter instructors' preferences manually:

In [ ]:
# instructors' preferred shift times

preferences = {}
hardcode_or_user_input = input("would you like to enter instructors' preferences manually (M) or use hardcoded data (H)? Warning: with eight instructors it is 600 data points.")

if hardcode_or_user_input == "M":

  print("is instructor y available to work shift x? 0-5 scale: 0 if not, 1 if not preferred, 5 if highly preferred")
  for index_3 in range(len(y)):
    for index_2 in range(5):
      for index_3 in range(len(j)):

        shift = x[index_2][index_3]

        # inputting instructors' preferences
        preference_input = input("is " + y[index_1] + " available to work " + shift + "?")
        preferences[(x[index_2][index_3], y[index_1])] = int(preference_input)

elif hardcode_or_user_input == "H":
  preferences = hardcoded_preferences
else:
  print("Please enter 'M' or 'H' with no quotes, spaces, or brackets.")

print(preferences)

would you like to enter instructors' preferences manually (M) or use hardcoded data (H)? Warning: with eight instructors it is 600 data points.H
{('M9:00', 'AA'): 1, ('M9:30', 'AA'): 1, ('M10:00', 'AA'): 3, ('M10:30', 'AA'): 3, ('M11:00', 'AA'): 3, ('M11:30', 'AA'): 3, ('M12:00', 'AA'): 3, ('M12:30', 'AA'): 3, ('M1:00', 'AA'): 4, ('M1:30', 'AA'): 4, ('M2:00', 'AA'): 4, ('M2:30', 'AA'): 0, ('M3:00', 'AA'): 0, ('M3:30', 'AA'): 0, ('M4:00', 'AA'): 1, ('T9:00', 'AA'): 1, ('T9:30', 'AA'): 1, ('T10:00', 'AA'): 1, ('T10:30', 'AA'): 1, ('T11:00', 'AA'): 1, ('T11:30', 'AA'): 0, ('T12:00', 'AA'): 0, ('T12:30', 'AA'): 0, ('T1:00', 'AA'): 1, ('T1:30', 'AA'): 1, ('T2:00', 'AA'): 1, ('T2:30', 'AA'): 1, ('T3:00', 'AA'): 1, ('T3:30', 'AA'): 1, ('T4:00', 'AA'): 1, ('W9:00', 'AA'): 1, ('W9:30', 'AA'): 1, ('W10:00', 'AA'): 3, ('W10:30', 'AA'): 3, ('W11:00', 'AA'): 3, ('W11:30', 'AA'): 3, ('W12:00', 'AA'): 3, ('W12:30', 'AA'): 3, ('W1:00', 'AA'): 4, ('W1:30', 'AA'): 4, ('W2:00', 'AA'): 4, ('W2:30', 'AA'):

In [ ]:
schedule = {}
objective_func = 0

for index_1 in range(5):
  for index_2 in range(len(j)):
    for index_3 in range(len(y)):

      # creating variables representing if instructor y will be working shift x
      var_label = "(x"+str(index_1)+" "+str(index_2)+",y"+str(index_3)+")"
      var = pulp.LpVariable(var_label, cat="Binary")
      schedule[((x[index_1][index_2]), y[index_3])] = var

      # adding vars to objective function with preference coefficients
      coefficient = preferences[(x[index_1][index_2], y[index_3])]
      objective_func += coefficient * var

print(schedule)

{('M9:00', 'AA'): (x0_0,y0), ('M9:00', 'MA'): (x0_0,y1), ('M9:00', 'MA2'): (x0_0,y2), ('M9:00', 'DB'): (x0_0,y3), ('M9:00', 'LL'): (x0_0,y4), ('M9:00', 'PO'): (x0_0,y5), ('M9:00', 'DW'): (x0_0,y6), ('M9:00', 'LY'): (x0_0,y7), ('M9:00', 'Dummy'): (x0_0,y8), ('M9:30', 'AA'): (x0_1,y0), ('M9:30', 'MA'): (x0_1,y1), ('M9:30', 'MA2'): (x0_1,y2), ('M9:30', 'DB'): (x0_1,y3), ('M9:30', 'LL'): (x0_1,y4), ('M9:30', 'PO'): (x0_1,y5), ('M9:30', 'DW'): (x0_1,y6), ('M9:30', 'LY'): (x0_1,y7), ('M9:30', 'Dummy'): (x0_1,y8), ('M10:00', 'AA'): (x0_2,y0), ('M10:00', 'MA'): (x0_2,y1), ('M10:00', 'MA2'): (x0_2,y2), ('M10:00', 'DB'): (x0_2,y3), ('M10:00', 'LL'): (x0_2,y4), ('M10:00', 'PO'): (x0_2,y5), ('M10:00', 'DW'): (x0_2,y6), ('M10:00', 'LY'): (x0_2,y7), ('M10:00', 'Dummy'): (x0_2,y8), ('M10:30', 'AA'): (x0_3,y0), ('M10:30', 'MA'): (x0_3,y1), ('M10:30', 'MA2'): (x0_3,y2), ('M10:30', 'DB'): (x0_3,y3), ('M10:30', 'LL'): (x0_3,y4), ('M10:30', 'PO'): (x0_3,y5), ('M10:30', 'DW'): (x0_3,y6), ('M10:30', 'LY'): 

Now we define our constraints and add them to the model. We have three types of constraints:

1. Schedule building constraints (guaranteeing only one instructor works each shift): For all xij, the sum of (xij, yk) over k = 1, 2, … n equals 1

In [ ]:
schedule_constraint = 0
for index_1 in range(5):

  for index_2 in range(len(j)):
    shifts_to_be_added = 0

    for index_3 in range(len(y)):
      shifts_to_be_added += schedule[(x[index_1][index_2], y[index_3])]

    schedule_constraint = shifts_to_be_added == 1
    mlc_scheduling_model += schedule_constraint

print(mlc_scheduling_model)

# want (xij, y0) + (xij, y1) + ... + (xij, yk) = 1
# (xij, y0) is represented by schedule[(x[][],y[0])]

# vars will show up below when u add them to constraints. build the schedule w/ some loops so u get i*j (75) constraints.

mlc_schedule:
MAXIMIZE
None
SUBJECT TO
_C1: (x0_0,y0) + (x0_0,y1) + (x0_0,y2) + (x0_0,y3) + (x0_0,y4) + (x0_0,y5)
 + (x0_0,y6) + (x0_0,y7) + (x0_0,y8) = 1

_C2: (x0_1,y0) + (x0_1,y1) + (x0_1,y2) + (x0_1,y3) + (x0_1,y4) + (x0_1,y5)
 + (x0_1,y6) + (x0_1,y7) + (x0_1,y8) = 1

_C3: (x0_2,y0) + (x0_2,y1) + (x0_2,y2) + (x0_2,y3) + (x0_2,y4) + (x0_2,y5)
 + (x0_2,y6) + (x0_2,y7) + (x0_2,y8) = 1

_C4: (x0_3,y0) + (x0_3,y1) + (x0_3,y2) + (x0_3,y3) + (x0_3,y4) + (x0_3,y5)
 + (x0_3,y6) + (x0_3,y7) + (x0_3,y8) = 1

_C5: (x0_4,y0) + (x0_4,y1) + (x0_4,y2) + (x0_4,y3) + (x0_4,y4) + (x0_4,y5)
 + (x0_4,y6) + (x0_4,y7) + (x0_4,y8) = 1

_C6: (x0_5,y0) + (x0_5,y1) + (x0_5,y2) + (x0_5,y3) + (x0_5,y4) + (x0_5,y5)
 + (x0_5,y6) + (x0_5,y7) + (x0_5,y8) = 1

_C7: (x0_6,y0) + (x0_6,y1) + (x0_6,y2) + (x0_6,y3) + (x0_6,y4) + (x0_6,y5)
 + (x0_6,y6) + (x0_6,y7) + (x0_6,y8) = 1

_C8: (x0_7,y0) + (x0_7,y1) + (x0_7,y2) + (x0_7,y3) + (x0_7,y4) + (x0_7,y5)
 + (x0_7,y6) + (x0_7,y7) + (x0_7,y8) = 1

_C9: (x0_8,y0) + (x0_8,y1

Workload constraints:

In [ ]:
for index_3 in range(len(y)):
  y_vars = 0

  for index_1 in range(5):
    for index_2 in range(len(j)):

      y_vars += schedule[(x[index_1][index_2], y[index_3])]

  y_workload_constraint = y_vars == 2 * workload[index_3]
  mlc_scheduling_model += y_workload_constraint

Now we add our objective function to the model (I do this last as i've found it ensures puLP recognizes this as the objective function and not one of the constraints):

In [ ]:
mlc_scheduling_model += objective_func

Solving the model:

In [ ]:
# Solve the problem
status = mlc_scheduling_model.solve()

# Print the solution
print(pulp.LpStatus[status])
print(mlc_scheduling_model)
print("-----------------------------------")

# display values of all variables
for index_1 in range(5):
  for index_2 in range(len(j)):
    for index_3 in range(len(y)):

      value = pulp.value(schedule[(x[index_1][index_2], y[index_3])])
      var_label = "(x"+str(index_1)+" "+str(index_2)+",y"+str(index_3)+")"

      if value == 1:
        print("SHIFT: ", var_label, " = ", value)
      else:
        print(var_label, " = ", value)


Optimal
mlc_schedule:
MAXIMIZE
1*(x0_0,y0) + 3*(x0_0,y1) + 2*(x0_0,y2) + 3*(x0_0,y3) + 1*(x0_0,y5) + 4*(x0_0,y7) + 1*(x0_1,y0) + 3*(x0_1,y1) + 2*(x0_1,y2) + 3*(x0_1,y3) + 1*(x0_1,y5) + 4*(x0_1,y7) + 4*(x0_10,y0) + 3*(x0_10,y2) + 4*(x0_10,y4) + 4*(x0_10,y5) + 1*(x0_10,y6) + 2*(x0_10,y7) + 2*(x0_11,y1) + 3*(x0_11,y2) + 4*(x0_11,y3) + 4*(x0_11,y4) + 4*(x0_11,y5) + 1*(x0_11,y6) + 1*(x0_11,y7) + 2*(x0_12,y1) + 3*(x0_12,y2) + 4*(x0_12,y3) + 3*(x0_12,y4) + 4*(x0_12,y5) + 1*(x0_12,y6) + 1*(x0_12,y7) + 2*(x0_13,y1) + 1*(x0_13,y2) + 4*(x0_13,y3) + 3*(x0_13,y4) + 3*(x0_13,y5) + 1*(x0_13,y6) + 1*(x0_13,y7) + 1*(x0_14,y0) + 2*(x0_14,y1) + 1*(x0_14,y2) + 4*(x0_14,y3) + 3*(x0_14,y4) + 1*(x0_14,y6) + 1*(x0_14,y7) + 3*(x0_2,y0) + 3*(x0_2,y1) + 4*(x0_2,y2) + 3*(x0_2,y3) + 3*(x0_2,y4) + 4*(x0_2,y6) + 3*(x0_3,y0) + 2*(x0_3,y1) + 4*(x0_3,y2) + 3*(x0_3,y3) + 3*(x0_3,y4) + 4*(x0_3,y6) + 3*(x0_4,y0) + 2*(x0_4,y1) + 4*(x0_4,y2) + 4*(x0_4,y3) + 3*(x0_4,y4) + 4*(x0_4,y6) + 3*(x0_5,y0) + 4*(x0_5,y3) + 4*(x0_5,y6)